In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
from matplotlib import dates as mdate
import json, re, pymysql, datetime, pandas
from pandas.io.json import json_normalize
pandas.options.display.float_format = "{:.0f}".format
pandas.set_option('display.max_rows', None)
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

In [ ]:
class BabiloniaPriceDataAnalytics:
    # variables can be access externally
    CEAGESP={}
    def __init__(self, config_path):
        with open(config_path) as json_data_file:
            data = json.load(json_data_file)
            url = data['SQLALCHEMY_DATABASE_URI']
            self.__MUX_PORT_THRESHOLD = data['MUX_PORT_THRESHOLD']
        user, password, host, database = re.match('mysql://(.*?):(.*?)@(.*?)/(.*)', url).groups()
        self.__CONN = pymysql.connect(host=host,port=3306,user=user,passwd=password,db=database)
      
    def get_ceagesp_dates(self):
        query="""
            SELECT DATE
            FROM farmland.PRICES 
            WHERE SOURCE='CEAGESP';
            """
        dates = pandas.read_sql_query(query, self.__CONN)
        return dates
    def get_ceagesp_prices(self, date):
        query="""
            SELECT DATA 
            FROM farmland.PRICES 
            WHERE SOURCE='CEAGESP' 
                AND DATE = '{}';
            """.format(date)
        prices = pandas.read_sql_query(query, self.__CONN)
        return prices
    def get_product_price(self, product, product_type):
        dates = self.get_ceagesp_dates()
        data_str = json.loads(self.get_ceagesp_prices('2020-04-27')['DATA'][0])
        data = json_normalize(json.loads(data_str))
        price_str = data[(data['Produto'] == product) & (data['Classificacao']==product_type)]['Comun'].values[0]
        price = float( price_str.replace(',', '.'))
        return price

In [ ]:
bpda = BabiloniaPriceDataAnalytics('../../server/app/config.json')

In [ ]:
print(bpda.get_product_price('RUCULA', 'HIDROPONICA'))

In [ ]:
print(bpda.get_ceagesp_dates())